In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import scipy
import camb
import scipy.linalg
from camb import model, initialpower
from camb.dark_energy import DarkEnergyPPF, DarkEnergyFluid

In [ ]:
#Set up the ell range
camb_ell_min          = 2#30
camb_ell_max          = 5000
camb_ell_range        = camb_ell_max  - camb_ell_min 

#load test data
testing_samples=np.load('parametersamples/cos_pkc_T128.npy',allow_pickle=True)[10000:]

testing_data_vectors=np.load('datavectors/Planck128/cos_pkc_T128_TT_acc.npy',allow_pickle=True)[10000:,:4998]

In [ ]:
#set device
#I prefer using CPU for model usage after training
device = torch.device('cpu')

#set the number of layers and internal dimension of the ResMLP blocks
Nlayer=3
intdim=4

In [ ]:

class Supact(nn.Module):
    def __init__(self, in_size):
        super(Supact, self).__init__()
        
        self.gamma = nn.Parameter(torch.ones(in_size))
        self.beta = nn.Parameter(torch.zeros(in_size))

    def forward(self, x):
        inv = (1+torch.exp(-1*torch.mul(self.beta,x))).pow_(-1)
        fac = 1-self.gamma
        mult = self.gamma + torch.mul(inv,fac)
        return torch.mul(mult,x)

class Affine(nn.Module):
    def __init__(self):
        super(Affine, self).__init__()

        # This function is designed for the Neuro-network to learn how to normalize the data between
        # layers. we will initiate gains and bias both at 1 
        self.gain = nn.Parameter(torch.ones(1))

        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):

        return x * self.gain + self.bias


class ResBlock(nn.Module):
    def __init__(self, in_size, out_size):
        super(ResBlock, self).__init__()
        
        if in_size != out_size: 
            self.skip = nn.Linear(in_size, out_size, bias=False) # we don't consider this. remove?
        else:
            self.skip = nn.Identity()

        self.layer1 = nn.Linear(in_size, out_size)
        self.layer2 = nn.Linear(out_size, out_size)

        self.norm1 = Affine()
        self.norm2 = Affine()

        self.act1 = Supact(in_size)#nn.Tanh()#nn.ReLU()#
        self.act2 = Supact(in_size)#nn.Tanh()#nn.ReLU()#

    def forward(self, x):
        xskip = self.skip(x)

        o1 = self.act1(self.layer1(self.norm1(x)))
        o2 = self.act2(self.layer2(self.norm2(o1))) + xskip

        return o2


class ResMLP(nn.Module):

    def __init__(self, input_dim, output_dim, int_dim, N_layer):

        super(ResMLP, self).__init__()

        modules=[]

        # Def: we will set the internal dimension as multiple of 128 (reason: just simplicity)
        int_dim = int_dim * 128

        # Def: we will only change the dimension of the datavector using linear transformations  
        modules.append(nn.Linear(input_dim, int_dim))
        
        # Def: by design, a pure block has the input and output dimension to be the same
        for n in range(N_layer):
            # Def: This is what we defined as a pure MLP block
            # Why the Affine function?
            #   R: this is for the Neuro-network to learn how to normalize the data between layer
            modules.append(ResBlock(int_dim, int_dim))
            modules.append(Supact(int_dim))
        
        # Def: the transformation from the internal dimension to the output dimension of the
        #      data vector we intend to emulate
        
        modules.append(nn.Linear(int_dim, output_dim))
        modules.append(Affine())
        # NN.SEQUENTIAL is a PYTHORCH function DEFINED AT: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
        # This function stacks up layers in the modules-list in sequence to create the whole model
        self.resmlp =nn.Sequential(*modules)#

    def forward(self, x):
        #x is a cosmological parameter set you feed in the model
        out = self.resmlp(x)

        return out



In [ ]:
#set up and load models
model = ResMLP(input_dim=9,output_dim=camb_ell_range,int_dim=intdim,N_layer=Nlayer)
model.to(device)
model = nn.DataParallel(model)

In [ ]:
covinv=np.load('cosvarinvTT.npy',allow_pickle=True)[:camb_ell_range,:camb_ell_range]


In [ ]:
PATH = "./trainedemu5000plkres/chiTTAstauresrescaleplk96i"+str(intdim)+'l'+str(Nlayer)
extrainfo=np.load("extra/extrainfo_plk_tt_third_T256_96.npy",allow_pickle=True)
X_mean=torch.Tensor(extrainfo.item()['X_mean']).to(device)
X_std=torch.Tensor(extrainfo.item()['X_std']).to(device)
Y_mean=torch.Tensor(extrainfo.item()['Y_mean']).to(device)
Y_std=torch.Tensor(extrainfo.item()['Y_std']).to(device)
Y_mean2=torch.Tensor(extrainfo.item()['Y_mean2']).to(device)
Y_std2=torch.Tensor(extrainfo.item()['Y_std2']).to(device)
model.load_state_dict(torch.load(PATH+'.pt',map_location=device))
model.eval()

In [ ]:
def predict(X):
    with torch.no_grad():
        X_norm=((X.float() - X_mean.float()) / X_std.float())
        X_norm[:,6:]=0
        #print(X_norm)
        pred=model(X_norm)
        #print(pred.dtype)
        
        M_pred=pred.to(device)
        y_pred = (M_pred.float() *Y_std2.float()+Y_mean2.float()).numpy()
        
    return y_pred
transform_matrix=np.load('extra/PCAmat_T256_tt_third.npy',allow_pickle=True)#PCA matrix loaded
def unnormalize(y_pred,X):
    y_pred=np.dot(d,transform_matrix)*Y_std.cpu().float().numpy()+Y_mean.cpu().float().numpy()
    for i in range(len(y_pred)):
        y_pred[i]=y_pred[i]*(np.exp(X[i,5].cpu().float().numpy()))/(np.exp(2*X[i,3].cpu().float().numpy()))
    return y_pred

#just a small trial to see if the pipline works
testing_samples=torch.Tensor(testing_samples)

testing_results=predict(testing_samples)

pred=unnormalize(testing_results,testing_samples)

In [ ]:
#computing chi2
diff=testing_data_vectors- pred

loss1 = np.diag(diff@covinv@diff.T)


print(r'mean $\chi^2=$',np.mean(loss1))
print(r'median $\chi^2=$',np.median(loss1))

In [ ]:
print(r'$f(\chi^2>1)=$',len([1 for i in loss1 if i > 1])/len(loss1))

In [ ]:
print(r'$f(\chi^2>0.2)=$',len([1 for i in loss1 if i > 0.2])/len(loss1))